## Example of training/testing using deterministic models with pytorch-lightning

In [1]:
%load_ext lab_black

In [2]:
from pathlib import Path
from types import SimpleNamespace
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from bnnrul.cmapss.dataset import CMAPSSDataModule
from bnnrul.cmapss.models import CMAPSSModel, get_checkpoint, TBLogger

In [3]:
args = SimpleNamespace(
    data_path="../data/cmapss/",
    out_path="../results/cmapss/",
    scn="dnn_ptl",
    net="linear",
)
checkpoint_dir = Path(f"{args.out_path}/{args.scn}/checkpoints/{args.net}")
logger = TBLogger(
    f"{args.out_path}/{args.scn}/lightning_logs/{args.net}",
    default_hp_metric=False,
)

In [4]:
# Just to illustrate. To properly train see scripts/cmapss_training.py
data = CMAPSSDataModule(args.data_path, batch_size=10000)
dnn = CMAPSSModel(data.win_length, data.n_features, args.net)

checkpoint_file = get_checkpoint(checkpoint_dir)
monitor = "loss/val"
checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_dir, monitor=monitor)
earlystopping_callback = EarlyStopping(monitor=monitor, patience=20)

trainer = pl.Trainer(
    gpus=[0],
    max_epochs=5,
    log_every_n_steps=2,
    resume_from_checkpoint=checkpoint_file,
    logger=logger,
    callbacks=[
        checkpoint_callback,
        earlystopping_callback,
    ],
)
trainer.fit(dnn, data)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Missing logger folder: ../results/cmapss//dnn_ptl/lightning_logs/linear/default

  | Name | Type   | Params
--------------------------------
0 | net  | Linear | 45.6 K
--------------------------------
45.6 K    Trainable params
0         Non-trainable params
45.6 K    Total params
0.182     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [5]:
data = CMAPSSDataModule(args.data_path, batch_size=1000)
dnn = CMAPSSModel.load_from_checkpoint(get_checkpoint(checkpoint_dir))
trainer = pl.Trainer(gpus=[0], log_every_n_steps=10, logger=logger)
trainer.test(dnn, data, verbose=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Testing: 0it [00:00, ?it/s]

[{'loss/test': 5659.830078125}]